# Load, merge and wrangle Oracle OpenWorld & CodeOne Session Data - Phase 1

The session catalog data is available from a REST API whose root is https://events.rainfocus.com/api/search. Check out Notebook OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb on how to fetch that data into (raw) local files

This notebook describes how to load, combine and wrangle the data from these files. This notebook shows for example how to load and merge data from dozens of (same formatted) JSON files, discard undesired attributes, deduplicate the record set, derive attributes for easier business intelligence & machine learning and write the resulting data set to a single JSON file.

## Load and merge data from raw JSON files
This first section describes how the session data from Oracle OpenWorld 2018 is loaded from over 30 individual files with the raw JSON session data. These files are organized by session type and event (oow and codeone) - and have been produced by a different notebook (OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb) from the Oracle OpenWorld Session Catalog API.

The files are read into individual Pandas Data Frame objects. These Data Frames are concatenated. The end result from reading over 30 files is a single Pandas Data Frame - called *ss* (session schedule).

In [158]:
#read a single session data file and parse the JSON content into a Pandas Data Frame
import pandas as pd
import json

#as a test, try to load data from one of the generated files 
ss = pd.read_json("oow2018-sessions_oow_TRN.json")
ss = ss.assign(catalog='oow')
ss.head(5)


abbreviation                                           abstract  \
0        TRN4070  What if you could benefit from new database qu...   
1        TRN4026  How does a real-world performance engineer int...   
10       TRN4567  Understand the tools and techniques available ...   
100      TRN6363  In order to derive value from the Internet of ...   
101      TRN5828  Oracle Linux powers tens of thousands of enter...   

     allowDoubleBooking                                    attributevalues  \
0                     0  [{'value': 'Beginner', 'attributevalue_id': '1...   
1                     0  [{'value': 'Beginner', 'attributevalue_id': '1...   
10                    0  [{'value': 'Beginner', 'attributevalue_id': '1...   
100                   0  [{'value': 'Product Training Session', 'attrib...   
101                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   

        code                              codeParts  code_id  \
0    TRN4070  {'alpha0': 'TRN', 'numeric1': '4070'}  trn4070   
1    TRN4026  {'alpha0': 'TRN', 'numeric1': '4026'}  trn4026   
10   TRN4567  {'alpha0': 'TRN', 'numeric1': '4567'}  trn4567   
100  TRN6363  {'alpha0': 'TRN', 'numeric1': '6363'}  trn6363   
101  TRN5828  {'alpha0': 'TRN', 'numeric1': '5828'}  trn5828   

           es_metadata_id             event eventCode   ...     \
0    1523650128750001VI1i  Oracle OpenWorld     oow18   ...      
1    1523576222383001pdRl  Oracle OpenWorld     oow18   ...      
10   1524514304383001hANs  Oracle OpenWorld     oow18   ...      
100  15321279159350012tnD  Oracle OpenWorld     oow18   ...      
101  1526317377636001I2Ll  Oracle OpenWorld     oow18   ...      

    type_displayorder_string useDoubleBooking useWaitingList  videos  \
0                       9999             True              0      []   
1                       9999             True              0      []   
10                      9999             True              0      []   
100                     9999             True              0      []   
101                     9999             True              0      []   

                                            viewAccess viewAccessPublic  \
0    [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
1    [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
10   [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
100  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
101  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   

     viewFileAccess                                waitlistAccess  \
0                []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
1                []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
10               []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
100              []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
101              []  [1533141196635001EZdJ, 1533143193464001uQsQ]   

    waitlistLimit  catalog  
0               0      oow  
1               0      oow  
10              0      oow  
100             0      oow  
101             0      oow  

[5 rows x 39 columns]

In [159]:
#as a test, try to load data from another file  
ss2 = pd.read_json("oow2018-sessions_codeone_TRN.json")
ss2 = ss2.assign(catalog='codeone')

ss2.head(5)


abbreviation                                           abstract  \
0       TRN4631  This session takes a deep dive into how the Or...   
1       TRN4081  Real-time data is an increasingly popular topi...   
10      TRN4084  .NET Core allows .NET applications to run cros...   
11      TRN4065  Zero downtime or the ability to sustain planne...   
2       TRN5774  Enterprises need to rapidly react to the chang...   

    allowDoubleBooking                                    attributevalues  \
0                    0  [{'value': 'Beginner', 'attributevalue_id': '1...   
1                    0  [{'value': 'Beginner', 'attributevalue_id': '1...   
10                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
11                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
2                    0  [{'value': 'Beginner', 'attributevalue_id': '1...   

       code                              codeParts  code_id  \
0   TRN4631  {'alpha0': 'TRN', 'numeric1': '4631'}  trn4631   
1   TRN4081  {'alpha0': 'TRN', 'numeric1': '4081'}  trn4081   
10  TRN4084  {'alpha0': 'TRN', 'numeric1': '4084'}  trn4084   
11  TRN4065  {'alpha0': 'TRN', 'numeric1': '4065'}  trn4065   
2   TRN5774  {'alpha0': 'TRN', 'numeric1': '5774'}  trn5774   

          es_metadata_id             event eventCode   ...     \
0   1524589840877001mdKy  Oracle OpenWorld     oow18   ...      
1   1523652232625001KxHD  Oracle OpenWorld     oow18   ...      
10  1523652635507001BgE3  Oracle OpenWorld     oow18   ...      
11  1523646070146001VKUN  Oracle OpenWorld     oow18   ...      
2   1526173783263001pwyK  Oracle OpenWorld     oow18   ...      

   type_displayorder_string useDoubleBooking useWaitingList  videos  \
0                      9999             True              0    None   
1                      9999             True              0      []   
10                     9999             True              0      []   
11                     9999             True              0      []   
2                      9999             True              0      []   

                                           viewAccess viewAccessPublic  \
0   [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
1   [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
10  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
11  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
2   [1533140082245001une2, 1533141196635001EZdJ, 1...             True   

    viewFileAccess                                waitlistAccess  \
0               []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
1               []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
10              []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
11              []  [1533141196635001EZdJ, 1533143193464001uQsQ]   
2               []  [1533141196635001EZdJ, 1533143193464001uQsQ]   

   waitlistLimit  catalog  
0              0  codeone  
1              0  codeone  
10             0  codeone  
11             0  codeone  
2              0  codeone  

[5 rows x 38 columns]

In [160]:
#add ss and ss2 together
#see https://pandas.pydata.org/pandas-docs/stable/merging.html
ss = pd.concat([ss,ss2],  ignore_index=True)
ss.head(8)

abbreviation                                           abstract  \
0      TRN4070  What if you could benefit from new database qu...   
1      TRN4026  How does a real-world performance engineer int...   
2      TRN4567  Understand the tools and techniques available ...   
3      TRN6363  In order to derive value from the Internet of ...   
4      TRN5828  Oracle Linux powers tens of thousands of enter...   
5      TRN4073  What do Oracle Database listener, dedicated se...   
6      TRN4022  Oracle Real Application Clusters (Oracle RAC) ...   
7      TRN4219  For more than 20 years, Oracle customers have ...   

   allowDoubleBooking                                    attributevalues  \
0                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
1                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
2                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
3                   0  [{'value': 'Product Training Session', 'attrib...   
4                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
5                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
6                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   
7                   0  [{'value': 'Beginner', 'attributevalue_id': '1...   

  catalog     code                              codeParts  code_id  \
0     oow  TRN4070  {'alpha0': 'TRN', 'numeric1': '4070'}  trn4070   
1     oow  TRN4026  {'alpha0': 'TRN', 'numeric1': '4026'}  trn4026   
2     oow  TRN4567  {'alpha0': 'TRN', 'numeric1': '4567'}  trn4567   
3     oow  TRN6363  {'alpha0': 'TRN', 'numeric1': '6363'}  trn6363   
4     oow  TRN5828  {'alpha0': 'TRN', 'numeric1': '5828'}  trn5828   
5     oow  TRN4073  {'alpha0': 'TRN', 'numeric1': '4073'}  trn4073   
6     oow  TRN4022  {'alpha0': 'TRN', 'numeric1': '4022'}  trn4022   
7     oow  TRN4219  {'alpha0': 'TRN', 'numeric1': '4219'}  trn4219   

         es_metadata_id             event      ...      type_displayorder  \
0  1523650128750001VI1i  Oracle OpenWorld      ...                   9999   
1  1523576222383001pdRl  Oracle OpenWorld      ...                   9999   
2  1524514304383001hANs  Oracle OpenWorld      ...                   9999   
3  15321279159350012tnD  Oracle OpenWorld      ...                   9999   
4  1526317377636001I2Ll  Oracle OpenWorld      ...                   9999   
5  1523650870534001B9V8  Oracle OpenWorld      ...                   9999   
6  1523572455776001BxBi  Oracle OpenWorld      ...                   9999   
7  152392541435100177jo  Oracle OpenWorld      ...                   9999   

  type_displayorder_string useDoubleBooking useWaitingList  videos  \
0                     9999             True              0      []   
1                     9999             True              0      []   
2                     9999             True              0      []   
3                     9999             True              0      []   
4                     9999             True              0      []   
5                     9999             True              0    None   
6                     9999             True              0      []   
7                     9999             True              0      []   

                                          viewAccess viewAccessPublic  \
0  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
1  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
2  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
3  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
4  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
5  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
6  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   
7  [1533140082245001une2, 1533141196635001EZdJ, 1...             True   

   viewFileAccess                                waitlistAccess waitlistLimit  
0

Overhead, two files were loaded, parsed into a Data Frame and added together into a single Data Frame. The next step is to load the session data from the raw JSON file for all files - for the two events and for all session types. The session types are defined in the Dict object *sessionTypes* . The code loops over the keys in this Dict and reads the corresponding JSON file for each of the two events.

In [161]:
sessionTypes =  {'BOF': '1518466139979001dQkv'
    , 'BQS': 'bqs'
    , 'BUS': '1519240082595001EpMm'
    , 'CAS': 'casestudy'
    , 'DEV': '1522435540042001BxTD'
    , 'ESS': 'ess'
    , 'FLP': 'flp'
    , 'GEN': 'general'
    , 'HOL': 'hol'
    , 'HOM': 'hom'
    , 'IGN': 'ignite'
    , 'KEY': 'option_1508950285425'
    , 'MTE':'1523906206279002QAu9'
    , 'PKN': '1527614217434001RBfj'
    , 'PRO': '1518464344082003KVWZ'
    , 'PRM': '1518464344082002KM3k'
    , 'TRN': '1518464344082001KHky'
    , 'SIG': 'sig'
    , 'THT': 'ts'
    , 'TLD': '1537894888625001RriS'
    , 'TIP': '1517517756579001F3CR'
    , 'TUT': 'tutorial'
#commented out because TRN is dealt with earlier on, 'TRN': '1518464344082001KHky'
}
#loop over all session types and read the corresponding files for both events codeone and oow
for key,value in sessionTypes.items():
    ssoow = pd.read_json("oow2018-sessions_oow_{0}.json".format(key))
    ssoow = ssoow.assign(catalog='oow')
    sscodeone = pd.read_json("oow2018-sessions_codeone_{0}.json".format(key))
    sscodeone = sscodeone.assign(catalog='codeone')
    ss = pd.concat([ss,ssoow,sscodeone],  ignore_index=True)
    

## Discard unneeded attributes
Early inspection of the JSON document and the session catalog website has provided insight in the available attributes and their relevance. Below, you will see an overview of all columns in the Data Frame - corresponding with the top level items in the JSON document. The subsequent step removes from the Data Frame all columns that seem irrelevant for our task at hand. 

These column seem relevant for the web frontend developers, for planned but not realized objectives or for unknown purposes. In order to not carry more weight than necessary - because of performance, resource usage and lurking complexity we get rid of columns that seem irrelevant. If we need them after all, they are still available in the data lake.    

Remove the unwanted columns from the Dataframe
(allowDoubleBooking .'codeParts', 'code_id', 'es_metadata_id',type_displayorder	type_displayorder_string	useDoubleBooking	useWaitingList	videos	viewAccess	viewAccessPublic, ...)


In [162]:
#list all columns in the Dataframe
ss.columns

Index(['abbreviation', 'abstract', 'allowDoubleBooking', 'attributevalues',
       'catalog', 'code', 'codeParts', 'code_id', 'es_metadata_id', 'event',
       'eventCode', 'eventId', 'eventName', 'externalID', 'featured_value',
       'files', 'highlight', 'length', 'participants',
       'publicViewPrivateSchedule', 'published', 'scheduleAccess', 'sessionID',
       'sponsors', 'status', 'times', 'title', 'title_sort', 'type',
       'type_displayorder', 'type_displayorder_string', 'useDoubleBooking',
       'useWaitingList', 'videos', 'viewAccess', 'viewAccessPublic',
       'viewFileAccess', 'waitlistAccess', 'waitlistLimit'],
      dtype='object')

In [163]:
# remove columns
# Note that the original 'data' object is changed when inplace=True
ss.drop([
    'allowDoubleBooking'
         ,'codeParts', 'code_id', 'es_metadata_id','type_displayorder'
       ,'type_displayorder_string', 'useDoubleBooking'
       ,'useWaitingList', 'videos'
      , 'viewAccess', 'viewAccessPublic','viewFileAccess', 'waitlistAccess', 'waitlistLimit'
    ,  'eventId','eventName','featured_value','publicViewPrivateSchedule','published', 'scheduleAccess','sessionID','status'
    ,'externalID','highlight','abbreviation'
        ]
        , axis=1, inplace=True) 

## Exploring the Data
Let's briefly look at the data we now have in the Pandas Data Frame. What does the data look like? What values do we have in the columns? And what complexity is hiding in some columns with nested values, such as participants, attributevalues and files.

Note: https://morphocode.com/pandas-cheat-sheet/ provides a quick overview of commands used for inspecting and manipulating the data frame.

In [164]:
#an overview of the current contents of the data frame
ss.head(6)


abstract  \
0  What if you could benefit from new database qu...   
1  How does a real-world performance engineer int...   
2  Understand the tools and techniques available ...   
3  In order to derive value from the Internet of ...   
4  Oracle Linux powers tens of thousands of enter...   
5  What do Oracle Database listener, dedicated se...   

                                     attributevalues catalog     code  \
0  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4070   
1  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4026   
2  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4567   
3  [{'value': 'Product Training Session', 'attrib...     oow  TRN6363   
4  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN5828   
5  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4073   

              event eventCode  \
0  Oracle OpenWorld     oow18   
1  Oracle OpenWorld     oow18   
2  Oracle OpenWorld     oow18   
3  Oracle OpenWorld     oow18   
4  Oracle OpenWorld     oow18   
5  Oracle OpenWorld     oow18   

                                               files  length  \
0  [{'fileId': '1540594881063001PjdE', 'filename'...    45.0   
1  [{'fileId': '1540053090863001JeFM', 'filename'...    45.0   
2  [{'fileId': '1539988618000001VcCe', 'filename'...    45.0   
3  [{'fileId': '1539982921144001RL4h', 'filename'...    45.0   
4  [{'fileId': '1540152941675001kZHw', 'filename'...    45.0   
5                                               None    45.0   

                                        participants sponsors  \
0  [{'speakerId': '14574694377280015QNA_150393630...     None   
1  [{'speakerId': '145746969078000151fg_150393630...     None   
2  [{'speakerId': '1468532302119001DtNf_150393630...     None   
3  [{'speakerId': '1500580326390001qTtd_150393630...     None   
4  [{'speakerId': '14574694616420015a7T_150393630...     None   
5  [{'speakerId': '14574694719240015Rek_150393630...     None   

                                               times  \
0  [{'sessionTimeID': '1532616475210001usPa', 'se...   
1  [{'sessionTimeID': '1531945554973001z4Hi', 'se...   
2  [{'sessionTimeID': '15352384203600012G04', 'se...   
3  [{'sessionTimeID': '1535223010412001BrNh', 'se...   
4  [{'sessionTimeID': '1535040668257001twFq', 'se...   
5  [{'sessionTimeID': '1531848510829001f1BX', 'se...   

                                               title  \
0  A Database Proxy for Transparent High Availabi...   
1  A Day in the Life of a Real-World Performance ...   
2  Best Practices for Diagnosing and Troubleshoot...   
3  Oracle Internet of Things Cloud Service and En...   
4  Oracle Linux and Oracle VM: Get Trained for Cl...   
5  Oracle Net Services: Best Practices for Databa...   

                       title_sort                      type  
0  adatabaseproxyfortransparenthi  Product Training Session  
1  adayinthelifeofarealworldperfo  Product Training Session  
2  bestpracticesfordiagnosingandt  Product Training Session  
3  oracleinternetofthingscloudser  Product Training Session  
4  oraclelinuxandoraclevmgettrain  Product Training Session  
5  oraclenetservicesbestpractices  Product Training Session

In [165]:
# and this overview of the rows and columns in the data frame.
print(ss.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 14 columns):
abstract           2339 non-null object
attributevalues    2339 non-null object
catalog            2339 non-null object
code               2339 non-null object
event              2339 non-null object
eventCode          2339 non-null object
files              1615 non-null object
length             2339 non-null float64
participants       2339 non-null object
sponsors           244 non-null object
times              2338 non-null object
title              2339 non-null object
title_sort         2339 non-null object
type               2339 non-null object
dtypes: float64(1), object(13)
memory usage: 255.9+ KB
None


In [166]:
#let's look at all different values for length (the duration of each session in minutes)
ss['length'].unique()
# yes - it is that simple!

array([  45.,   20.,  240.,  180.,   90.,  120.,   75.,   60.,   30.,
        105.,   50.,   25.,   55.,   15.])

In [167]:
# and what about (session) type?
ss['type'].unique()

array(['Product Training Session', 'Birds of a Feather (BOF) Session',
       'Burning Questions Session', 'Business Use Case Session',
       'Customer Case Study Session', 'Developer Session',
       'Executive Solution Session', 'Flipped Session', 'General Session',
       'HOL (Hands-on Lab) Session', 'Home Room Session', 'Ignite Session',
       'Keynote Session', 'Meet the Experts', 'Product Keynote Session',
       'Product Overview Session', 'Product Roadmap Session',
       'Special Interest Group (SIG) Meeting', 'Theater Session',
       'Thought Leader Session', 'Tips and Tricks Session',
       'Tutorial Session'], dtype=object)

In [168]:
#some of the columns or attributes have nested values. It seems useful to take a closer look at them. 
#show nested array attributevalues for first record in Dataframe - this is an attribute which contains an array of objects that each define a specific characteristic of the session, 
# such as date and time, track, topic, level, role, company market size, 

ss['attributevalues'][100]

[{'attribute': 'Session Type',
  'attribute_id': 'SessionType',
  'attributevalue_code': '1518464344082001KHky',
  'attributevalue_id': '1518464344082001KHky',
  'dataForm': '1503936308L8ef50df64',
  'displayorder': 26.0,
  'published': 1.0,
  'rf_attributevalue_id': '1518464344506001KUj0',
  'scheduleAccess': ['1533141196635001EZdJ', '1533143193464001uQsQ'],
  'value': 'Product Training Session',
  'viewAccess': ['1533140082245001une2',
   '1533141196635001EZdJ',
   '1533143193464001uQsQ',
   '1533146325902001u9v6'],
  'viewAccessPublic': True},
 {'attribute': 'Your Cloud Success: Training and End to End Support',
  'attribute_id': 'YourCloudSuccessTrainingandEndtoEndSupport',
  'attributevalue_code': '1532623698924002WFNS',
  'attributevalue_id': '1532623698924002WFNS',
  'dataForm': '1503936308L8ef50df64',
  'displayorder': 1.0,
  'published': 1.0,
  'rf_attributevalue_id': '1532623701201001WXTO',
  'scheduleAccess': ['1533141196635001EZdJ', '1533143193464001uQsQ'],
  'value': 'Your

In [169]:
#show nested array participants for first record in Dataframe
ss['participants'][10]


[{'Speaker-Photo-Published': 'Published',
  'attributevalues': [{'attribute': 'Special Designations',
    'attribute_id': 'specialdesignations',
    'attributevalue_id': 'sdoraclecertified',
    'dataForm': '1503936308F8c7e51a2e',
    'value': 'Oracle Certified'},
   {'attribute': 'Special Designations',
    'attribute_id': 'specialdesignations',
    'attributevalue_id': 'sdjavaonerockstar',
    'dataForm': '1503936308F8c7e51a2e',
    'value': 'JavaOne Rockstar'}],
  'bio': 'My name is Ron Soltani, and I am a Sr. Principal Instructor at Oracle University.  I have been with Oracle University for over 14 Years teaching, Oracle Database, MYSQL, and Times Ten Database courses and I am the lead instructor for Oracle Database New Features, Oracle Database Security, and MYSQL Administration courses.  I am based in Southern Oregon and enjoy spending my spare time outdoors and working on our farm. ',
  'companyName': 'Oracle',
  'displayorder': 0.0,
  'es_metadata_id': '1500578268510001KsST_150

In [170]:
#show nested array files
ss['files'][0]

[{'fileId': '1540594881063001PjdE',
  'filename': 'TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security.pdf',
  'url': 'https://static.rainfocus.com/oracle/oow18/sess/1523650128750001VI1i/PF/TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security_1540594881049001PwHW.pdf',
  'viewFileAccess': []}]

## Enriching the data
In this section we are engineering the data to produce some attributes or features that are easier to work with once we start doing data science activities such as business intelligence or machine learning. In this section we are not yet actually bringing in external data sources to add information we not already have in our set. However, we are making the data we already have more accessible and thereby more valuable. So in that sense, this too can be called *enriching*. 

The enrichments performed on the session data in the data frame :
* set a flag at session level to indicate whether the session is an Oracle session (with Oracle staff among the speakers- speaker has Oracle in companyName attribute)
* set a flag at session level to indicate whether a file has been uploaded for the session
* set a flag at session level to indicate whether one of the speakers has Java Rockstar, Java Champion, Developer Champion/Groundbreaker Ambassador, ACE or ACE Director
* set attribute level (based on attributevalues array) - beginner, all, ...
* derive track(s?) from attributevalues array (where attribute_id=Track or attribute_id=CodeOneTracks  )
* set attribute with number of speakers on session
* number of instances of the session    

In [171]:
#function to derive Oracle flag from participants
def oracle_speaker(session):
      result = "N"
      # loop over speakers; if for any of them, companyName contains the word Oracle, the result =Y  
      for x in session["participants"][:]:
        if ("oracle" in x.get('companyName','x').lower()):
          result='Y'
      return result

In [172]:
#function to derive file flag from files
def file_flag(session):
      result = "N"
      if type(session.get("files",None)) =='list' :
          # loop over files; if any exist, then result = Y
          for x in session["files"][:]:
             if x['fileId']:
               result='Y'
      return result

In [173]:
#set oracle_speaker flag
#apply function oracle_speaker to every row in the data frame to derive values for the new column oracle_speaker
ss['oracle_speaker'] = ss.apply(oracle_speaker, axis=1)

#set file_flag
#apply function file_flag to every row in the data frame to derive values for the new column file_flag
ss['file_flag'] = ss.apply(file_flag, axis=1)

#show sessions 
ss.tail(5)

abstract  \
2334  Java is moving faster and faster, and a lot of...   
2335  You’re a Java developer? Already familiar with...   
2336  Java 9 introduced JShell as a REPL tool, and v...   
2337  During this tutorial, you will learn how to us...   
2338  Are you tired of null pointer exceptions, unin...   

                                        attributevalues  catalog     code  \
2334  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4828   
2335  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5930   
2336  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5297   
2337  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT6303   
2338  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4991   

                 event eventCode  \
2334   Oracle Code One     oow18   
2335   Oracle Code One     oow18   
2336   Oracle Code One     oow18   
2337  Oracle OpenWorld     oow18   
2338   Oracle Code One     oow18   

                                                  files  length  \
2334  [{'fileId': '1540086019771001k4LD', 'filename'...   120.0   
2335  [{'fileId': '1540272055239001vAaw', 'filename'...   120.0   
2336                                               None   120.0   
2337  [{'fileId': '1541454340556001bnq5', 'filename'...   120.0   
2338  [{'fileId': '1540363608323001karC', 'filename'...   120.0   

                                           participants sponsors  \
2334  [{'speakerId': '1463271742329001nOLV_150393630...     None   
2335  [{'speakerId': '1492999456284001PEDn_150393630...     None   
2336  [{'speakerId': '14574694816540015LCz_150393630...     None   
2337  [{'speakerId': '1464284608830001XvOn_150393630...     None   
2338  [{'speakerId': '145746944456700158Jh_150393630...     None   

                                                  times  \
2334  [{'sessionTimeID': '153618822712200139zu', 'se...   
2335  [{'sessionTimeID': '15362149378730013qhw', 'se...   
2336  [{'sessionTimeID': '1536188167148001xmdi', 'se...   
2337  [{'sessionTimeID': '15354064474310011Ozp', 'se...   
2338  [{'sessionTimeID': '1535411051423001Bw5D', 'se...   

                                                  title  \
2334     Java 5, 6, 7, 8, 9, 10, 11: What Did You Miss?   
2335                     Kubernetes for Java Developers   
2336  Let's Start Java11 from the Beginning with JShell   
2337  MySQL 8.0 Document Store: Everything You Need ...   
2338  Preventing Errors Before They Happen: The Chec...   

                          title_sort              type oracle_speaker  \
2334     java567891011whatdidyoumiss  Tutorial Session              N   
2335     kubernetesforjavadevelopers  Tutorial Session              N   
2336  letsstartjava11fromthebeginnin  Tutorial Session              N   
2337  mysql80documentstoreeverything  Tutorial Session              Y   
2338  preventingerrorsbeforetheyhapp  Tutorial Session              N   

     file_flag  
2334         N  
2335         N  
2336         N  
2337         N  
2338         N

In [174]:
#function to derive designation flag from speakers
# values for designation: JavaOne Rockstar, Oracle ACE Director, Oracle Java Champion, Groundbreaker Ambassador, 
def speaker_designation(session, designation):
      result = "N"
      # loop over speakers and their attributevalues; if any exist with attribute_id == specialdesignations and value == JavaOne Rockstar
      for x in session["participants"][:]:
          if "attributevalues" in x: 
              for y in x["attributevalues"][:]:
                if "attribute_id" in y: 
                    if y["attribute_id"]=="specialdesignations":
                        if y["value"]== designation:
                            result="Y"
      return result

In [175]:
#set flags for designations
designations = ['JavaOne Rockstar', 'Oracle ACE Director', 'Oracle Java Champion', 'Groundbreaker Ambassador']
for d in designations:
   ss[d] = ss.apply(speaker_designation, args=(d,), axis=1) 

ss.head(5)

abstract  \
0  What if you could benefit from new database qu...   
1  How does a real-world performance engineer int...   
2  Understand the tools and techniques available ...   
3  In order to derive value from the Internet of ...   
4  Oracle Linux powers tens of thousands of enter...   

                                     attributevalues catalog     code  \
0  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4070   
1  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4026   
2  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4567   
3  [{'value': 'Product Training Session', 'attrib...     oow  TRN6363   
4  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN5828   

              event eventCode  \
0  Oracle OpenWorld     oow18   
1  Oracle OpenWorld     oow18   
2  Oracle OpenWorld     oow18   
3  Oracle OpenWorld     oow18   
4  Oracle OpenWorld     oow18   

                                               files  length  \
0  [{'fileId': '1540594881063001PjdE', 'filename'...    45.0   
1  [{'fileId': '1540053090863001JeFM', 'filename'...    45.0   
2  [{'fileId': '1539988618000001VcCe', 'filename'...    45.0   
3  [{'fileId': '1539982921144001RL4h', 'filename'...    45.0   
4  [{'fileId': '1540152941675001kZHw', 'filename'...    45.0   

                                        participants sponsors  \
0  [{'speakerId': '14574694377280015QNA_150393630...     None   
1  [{'speakerId': '145746969078000151fg_150393630...     None   
2  [{'speakerId': '1468532302119001DtNf_150393630...     None   
3  [{'speakerId': '1500580326390001qTtd_150393630...     None   
4  [{'speakerId': '14574694616420015a7T_150393630...     None   

                                               times  \
0  [{'sessionTimeID': '1532616475210001usPa', 'se...   
1  [{'sessionTimeID': '1531945554973001z4Hi', 'se...   
2  [{'sessionTimeID': '15352384203600012G04', 'se...   
3  [{'sessionTimeID': '1535223010412001BrNh', 'se...   
4  [{'sessionTimeID': '1535040668257001twFq', 'se...   

                                               title  \
0  A Database Proxy for Transparent High Availabi...   
1  A Day in the Life of a Real-World Performance ...   
2  Best Practices for Diagnosing and Troubleshoot...   
3  Oracle Internet of Things Cloud Service and En...   
4  Oracle Linux and Oracle VM: Get Trained for Cl...   

                       title_sort                      type oracle_speaker  \
0  adatabaseproxyfortransparenthi  Product Training Session              Y   
1  adayinthelifeofarealworldperfo  Product Training Session              Y   
2  bestpracticesfordiagnosingandt  Product Training Session              Y   
3  oracleinternetofthingscloudser  Product Training Session              Y   
4  oraclelinuxandoraclevmgettrain  Product Training Session              Y   

  file_flag JavaOne Rockstar Oracle ACE Director Oracle Java Champion  \
0         N                N                   N                    N   
1         N                N                   N                    N   
2         N                N                   N                    N   
3         N                N                   N                    N   
4         N                N                   N                    N   

  Groundbreaker Ambassador  
0                        N  
1                        N  
2                        N  
3                        N  
4                        N

In [176]:
#function to derive level flag for a session
def session_level(session, level):
      result = "N"
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="SessionsbyExperienceLevel":
                if x["value"]== level:
                    result="Y"
      return result

In [177]:
#set flags for designations
levels = ['Intermediate', 'Beginner', 'Advanced', 'All']
for l in levels:
   ss[l] = ss.apply(session_level, args=(l,), axis=1) 
print("Assigned Level Flags (Advanced, Intermediate, Beginner)")
ss.tail(5)

Assigned Level Flags (Advanced, Intermediate, Beginner)


abstract  \
2334  Java is moving faster and faster, and a lot of...   
2335  You’re a Java developer? Already familiar with...   
2336  Java 9 introduced JShell as a REPL tool, and v...   
2337  During this tutorial, you will learn how to us...   
2338  Are you tired of null pointer exceptions, unin...   

                                        attributevalues  catalog     code  \
2334  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4828   
2335  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5930   
2336  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5297   
2337  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT6303   
2338  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4991   

                 event eventCode  \
2334   Oracle Code One     oow18   
2335   Oracle Code One     oow18   
2336   Oracle Code One     oow18   
2337  Oracle OpenWorld     oow18   
2338   Oracle Code One     oow18   

                                                  files  length  \
2334  [{'fileId': '1540086019771001k4LD', 'filename'...   120.0   
2335  [{'fileId': '1540272055239001vAaw', 'filename'...   120.0   
2336                                               None   120.0   
2337  [{'fileId': '1541454340556001bnq5', 'filename'...   120.0   
2338  [{'fileId': '1540363608323001karC', 'filename'...   120.0   

                                           participants sponsors ...  \
2334  [{'speakerId': '1463271742329001nOLV_150393630...     None ...   
2335  [{'speakerId': '1492999456284001PEDn_150393630...     None ...   
2336  [{'speakerId': '14574694816540015LCz_150393630...     None ...   
2337  [{'speakerId': '1464284608830001XvOn_150393630...     None ...   
2338  [{'speakerId': '145746944456700158Jh_150393630...     None ...   

     oracle_speaker file_flag JavaOne Rockstar Oracle ACE Director  \
2334              N         N                N                   N   
2335              N         N                N                   N   
2336              N         N                N                   N   
2337              Y         N                N                   N   
2338              N         N                Y                   N   

     Oracle Java Champion Groundbreaker Ambassador Intermediate Beginner  \
2334                    Y                        Y            Y        Y   
2335                    N                        N            Y        Y   
2336                    N                        N            N        Y   
2337                    N                        N            Y        Y   
2338                    N                        N            N        Y   

     Advanced All  
2334        Y   N  
2335        N   N  
2336        N   N  
2337        Y   N  
2338        N   N  

[5 rows x 24 columns]

In [178]:
#function to derive track flag for a session
def session_track(session):

      result = ['abc']      
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="CodeOneTracks":
               result.append( x["value"])
            if x["attribute_id"]=="Track":               
               result.append( x["value"])
      del result[0]   
      return result

In [179]:
#set flags for designations
ss["track"] = ss.apply(session_track,  axis=1) 
print("Assigned Track")
ss.head(5)

Assigned Track


abstract  \
0  What if you could benefit from new database qu...   
1  How does a real-world performance engineer int...   
2  Understand the tools and techniques available ...   
3  In order to derive value from the Internet of ...   
4  Oracle Linux powers tens of thousands of enter...   

                                     attributevalues catalog     code  \
0  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4070   
1  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4026   
2  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4567   
3  [{'value': 'Product Training Session', 'attrib...     oow  TRN6363   
4  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN5828   

              event eventCode  \
0  Oracle OpenWorld     oow18   
1  Oracle OpenWorld     oow18   
2  Oracle OpenWorld     oow18   
3  Oracle OpenWorld     oow18   
4  Oracle OpenWorld     oow18   

                                               files  length  \
0  [{'fileId': '1540594881063001PjdE', 'filename'...    45.0   
1  [{'fileId': '1540053090863001JeFM', 'filename'...    45.0   
2  [{'fileId': '1539988618000001VcCe', 'filename'...    45.0   
3  [{'fileId': '1539982921144001RL4h', 'filename'...    45.0   
4  [{'fileId': '1540152941675001kZHw', 'filename'...    45.0   

                                        participants sponsors  \
0  [{'speakerId': '14574694377280015QNA_150393630...     None   
1  [{'speakerId': '145746969078000151fg_150393630...     None   
2  [{'speakerId': '1468532302119001DtNf_150393630...     None   
3  [{'speakerId': '1500580326390001qTtd_150393630...     None   
4  [{'speakerId': '14574694616420015a7T_150393630...     None   

                         ...                         file_flag  \
0                        ...                                 N   
1                        ...                                 N   
2                        ...                                 N   
3                        ...                                 N   
4                        ...                                 N   

  JavaOne Rockstar Oracle ACE Director Oracle Java Champion  \
0                N                   N                    N   
1                N                   N                    N   
2                N                   N                    N   
3                N                   N                    N   
4                N                   N                    N   

  Groundbreaker Ambassador Intermediate Beginner Advanced All  \
0                        N            Y        Y        Y   N   
1                        N            Y        Y        Y   N   
2                        N            Y        Y        Y   N   
3                        N            N        N        N   N   
4                        N            Y        Y        Y   N   

                                               track  
0  [- Application Development, Oracle Cloud Platf...  
1  [- Application Development, Oracle Cloud Platf...  
2  [SMB Solutions, Oracle Cloud Platform, - Cloud...  
3                                                 []  
4  [Your Cloud Transformation Roadmap, - Upgradin...  

[5 rows x 25 columns]

In [180]:
#set number of speakers for each session by taking the length of the list in the participants column
ss["speaker_count"] = ss['participants'].apply(lambda x: len(x))
ss.head(5)

abstract  \
0  What if you could benefit from new database qu...   
1  How does a real-world performance engineer int...   
2  Understand the tools and techniques available ...   
3  In order to derive value from the Internet of ...   
4  Oracle Linux powers tens of thousands of enter...   

                                     attributevalues catalog     code  \
0  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4070   
1  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4026   
2  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4567   
3  [{'value': 'Product Training Session', 'attrib...     oow  TRN6363   
4  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN5828   

              event eventCode  \
0  Oracle OpenWorld     oow18   
1  Oracle OpenWorld     oow18   
2  Oracle OpenWorld     oow18   
3  Oracle OpenWorld     oow18   
4  Oracle OpenWorld     oow18   

                                               files  length  \
0  [{'fileId': '1540594881063001PjdE', 'filename'...    45.0   
1  [{'fileId': '1540053090863001JeFM', 'filename'...    45.0   
2  [{'fileId': '1539988618000001VcCe', 'filename'...    45.0   
3  [{'fileId': '1539982921144001RL4h', 'filename'...    45.0   
4  [{'fileId': '1540152941675001kZHw', 'filename'...    45.0   

                                        participants sponsors      ...       \
0  [{'speakerId': '14574694377280015QNA_150393630...     None      ...        
1  [{'speakerId': '145746969078000151fg_150393630...     None      ...        
2  [{'speakerId': '1468532302119001DtNf_150393630...     None      ...        
3  [{'speakerId': '1500580326390001qTtd_150393630...     None      ...        
4  [{'speakerId': '14574694616420015a7T_150393630...     None      ...        

  JavaOne Rockstar Oracle ACE Director Oracle Java Champion  \
0                N                   N                    N   
1                N                   N                    N   
2                N                   N                    N   
3                N                   N                    N   
4                N                   N                    N   

  Groundbreaker Ambassador Intermediate Beginner Advanced All  \
0                        N            Y        Y        Y   N   
1                        N            Y        Y        Y   N   
2                        N            Y        Y        Y   N   
3                        N            N        N        N   N   
4                        N            Y        Y        Y   N   

                                               track speaker_count  
0  [- Application Development, Oracle Cloud Platf...             3  
1  [- Application Development, Oracle Cloud Platf...             2  
2  [SMB Solutions, Oracle Cloud Platform, - Cloud...             1  
3                                                 []             1  
4  [Your Cloud Transformation Roadmap, - Upgradin...             2  

[5 rows x 26 columns]

In [181]:
#set number of instancesfor each session by taking the length of the list in the times column
ss["instance_count"] = ss['times'].apply(lambda x: len(x) if x else None)
ss.tail(5)

abstract  \
2334  Java is moving faster and faster, and a lot of...   
2335  You’re a Java developer? Already familiar with...   
2336  Java 9 introduced JShell as a REPL tool, and v...   
2337  During this tutorial, you will learn how to us...   
2338  Are you tired of null pointer exceptions, unin...   

                                        attributevalues  catalog     code  \
2334  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4828   
2335  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5930   
2336  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT5297   
2337  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT6303   
2338  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  TUT4991   

                 event eventCode  \
2334   Oracle Code One     oow18   
2335   Oracle Code One     oow18   
2336   Oracle Code One     oow18   
2337  Oracle OpenWorld     oow18   
2338   Oracle Code One     oow18   

                                                  files  length  \
2334  [{'fileId': '1540086019771001k4LD', 'filename'...   120.0   
2335  [{'fileId': '1540272055239001vAaw', 'filename'...   120.0   
2336                                               None   120.0   
2337  [{'fileId': '1541454340556001bnq5', 'filename'...   120.0   
2338  [{'fileId': '1540363608323001karC', 'filename'...   120.0   

                                           participants sponsors  \
2334  [{'speakerId': '1463271742329001nOLV_150393630...     None   
2335  [{'speakerId': '1492999456284001PEDn_150393630...     None   
2336  [{'speakerId': '14574694816540015LCz_150393630...     None   
2337  [{'speakerId': '1464284608830001XvOn_150393630...     None   
2338  [{'speakerId': '145746944456700158Jh_150393630...     None   

          ...       Oracle ACE Director Oracle Java Champion  \
2334      ...                         N                    Y   
2335      ...                         N                    N   
2336      ...                         N                    N   
2337      ...                         N                    N   
2338      ...                         N                    N   

     Groundbreaker Ambassador Intermediate Beginner Advanced All  \
2334                        Y            Y        Y        Y   N   
2335                        N            Y        Y        N   N   
2336                        N            N        Y        N   N   
2337                        N            Y        Y        Y   N   
2338                        N            N        Y        N   N   

                                                  track speaker_count  \
2334                               [Core Java Platform]             1   
2335  [Containers, Serverless, and Cloud, Developmen...             1   
2336                               [Core Java Platform]             1   
2337            [Oracle Cloud Platform, - MySQL, MySQL]             1   
2338                                [Development Tools]             2   

     instance_count  
2334            1.0  
2335            1.0  
2336            1.0  
2337            1.0  
2338            1.0  

[5 rows x 27 columns]

In [182]:
#set the room capacity based on the capacity of the room of the first entry in the times list
# note: when the session is scheduled multiple times, not all rooms may have the same capacity; that detail gets lost
ss["room_capacity"]= ss['times'].apply(lambda x:x[0]['capacity'] if x else None)
#likewise derive day, time and room - from the first occurrence of the session
ss["day"]= ss['times'].apply(lambda x:x[0]['dayName'] if x else None)
ss["room"]= ss['times'].apply(lambda x:x[0]['room'] if x else None)
ss["time"]= ss['times'].apply(lambda x:x[0]['time'] if x else None)
ss.head(5)

abstract  \
0  What if you could benefit from new database qu...   
1  How does a real-world performance engineer int...   
2  Understand the tools and techniques available ...   
3  In order to derive value from the Internet of ...   
4  Oracle Linux powers tens of thousands of enter...   

                                     attributevalues catalog     code  \
0  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4070   
1  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4026   
2  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN4567   
3  [{'value': 'Product Training Session', 'attrib...     oow  TRN6363   
4  [{'value': 'Beginner', 'attributevalue_id': '1...     oow  TRN5828   

              event eventCode  \
0  Oracle OpenWorld     oow18   
1  Oracle OpenWorld     oow18   
2  Oracle OpenWorld     oow18   
3  Oracle OpenWorld     oow18   
4  Oracle OpenWorld     oow18   

                                               files  length  \
0  [{'fileId': '1540594881063001PjdE', 'filename'...    45.0   
1  [{'fileId': '1540053090863001JeFM', 'filename'...    45.0   
2  [{'fileId': '1539988618000001VcCe', 'filename'...    45.0   
3  [{'fileId': '1539982921144001RL4h', 'filename'...    45.0   
4  [{'fileId': '1540152941675001kZHw', 'filename'...    45.0   

                                        participants sponsors  ...   Beginner  \
0  [{'speakerId': '14574694377280015QNA_150393630...     None  ...          Y   
1  [{'speakerId': '145746969078000151fg_150393630...     None  ...          Y   
2  [{'speakerId': '1468532302119001DtNf_150393630...     None  ...          Y   
3  [{'speakerId': '1500580326390001qTtd_150393630...     None  ...          N   
4  [{'speakerId': '14574694616420015a7T_150393630...     None  ...          Y   

  Advanced All                                              track  \
0        Y   N  [- Application Development, Oracle Cloud Platf...   
1        Y   N  [- Application Development, Oracle Cloud Platf...   
2        Y   N  [SMB Solutions, Oracle Cloud Platform, - Cloud...   
3        N   N                                                 []   
4        Y   N  [Your Cloud Transformation Roadmap, - Upgradin...   

  speaker_count instance_count room_capacity        day  \
0             3            1.0           308  Wednesday   
1             2            1.0           313     Monday   
2             1            1.0           312     Monday   
3             1            1.0           173     Monday   
4             2            1.0         50000     Monday   

                       room   time  
0  Moscone West - Room 3009  11:15  
1  Moscone West - Room 3004  15:45  
2  Moscone West - Room 3006  11:30  
3  Moscone West - Room 3000  12:30  
4  Moscone South - Room 154  10:30  

[5 rows x 31 columns]

## Deduplicate
Some sessions are included in the catalog for both events - Oracle OpenWorld and CodeOne - even though they are associated with one of the two. The exact same session - with only a different value for attribute catalog - occurs twice in our data set for these sessions. We should get rid of duplicates. However, we should not do so before we capture the fact that a session is part of the catalogs of both events in the record that is retained. 

Note: there some sessions that occur multiple times in the data set but are not included in both event catalogs.

Let's first see how many events are part of both events' catalogs. 

In [183]:
#All three approaches do the work of finding all rows with code values that occur more than once

#ss[ss['code'].duplicated(keep=False)]
#ss.groupby('code').filter(lambda x: len(x) > 1)

counts = ss['code'].value_counts()
duplicates = ss[ss['code'].isin(counts.index[counts > 1])]

duplicates.sort_values('code').head(10)


abstract  \
147  Microservices are independent—sure. Complex tr...   
151  Microservices are independent—sure. Complex tr...   
148  Two years ago, after a painful history of wate...   
166  Two years ago, after a painful history of wate...   
149  The migration from monolith to microservices h...   
167  The migration from monolith to microservices h...   
146  Data is the essential fuel to every analytic p...   
150  Data is the essential fuel to every analytic p...   
361  Since Oracle announced support for some of its...   
290  Since Oracle announced support for some of its...   

                                       attributevalues  catalog     code  \
147  [{'value': 'Intermediate', 'attributevalue_id'...      oow  BOF4977   
151  [{'value': 'Intermediate', 'attributevalue_id'...  codeone  BOF4977   
148  [{'value': 'Beginner', 'attributevalue_id': '1...      oow  BOF5232   
166  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  BOF5232   
149  [{'value': 'Beginner', 'attributevalue_id': '1...      oow  BOF5714   
167  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  BOF5714   
146  [{'value': 'Beginner', 'attributevalue_id': '1...      oow  BOF6086   
150  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  BOF6086   
361  [{'value': 'Beginner', 'attributevalue_id': '1...  codeone  BUS1224   
290  [{'value': 'Beginner', 'attributevalue_id': '1...      oow  BUS1224   

                event eventCode  \
147  Oracle OpenWorld     oow18   
151  Oracle OpenWorld     oow18   
148  Oracle OpenWorld     oow18   
166  Oracle OpenWorld     oow18   
149  Oracle OpenWorld     oow18   
167  Oracle OpenWorld     oow18   
146  Oracle OpenWorld     oow18   
150  Oracle OpenWorld     oow18   
361  Oracle OpenWorld     oow18   
290  Oracle OpenWorld     oow18   

                                                 files  length  \
147  [{'fileId': '1540345180625001DgUR', 'filename'...    45.0   
151  [{'fileId': '1540345180625001DgUR', 'filename'...    45.0   
148  [{'fileId': '1540253608259001DfiR', 'filename'...    45.0   
166  [{'fileId': '1540253608259001DfiR', 'filename'...    45.0   
149  [{'fileId': '1540337005562001JXhu', 'filename'...    45.0   
167  [{'fileId': '1540337005562001JXhu', 'filename'...    45.0   
146  [{'fileId': '1540230846859001JH0Y', 'filename'...    45.0   
150  [{'fileId': '1540230846859001JH0Y', 'filename'...    45.0   
361  [{'fileId': '1537442351848001mIzP', 'filename'...    45.0   
290  [{'fileId': '1537442351848001mIzP', 'filename'...    45.0   

                                          participants sponsors  ...    \
147  [{'speakerId': '14574694420080015I8e_150393630...     None  ...     
151  [{'speakerId': '14574694420080015I8e_150393630...     None  ...     
148  [{'speakerId': '14574694740030015Ufr_150393630...     None  ...     
166  [{'speakerId': '14574694740030015Ufr_150393630...     None  ...     
149  [{'speakerId': '1526055148128001q2D3_150393630...     None  ...     
167  [{'speakerId': '1526055148128001q2D3_150393630...     None  ...     
146  [{'speakerId': '14574694443730025zo5_150393630...       []  ...     
150  [{'speakerId': '14574694443730025zo5_150393630...       []  ...     
361  [{'speakerId': '14574695992920015zm9_150393630...     None  ...     
290  [{'speakerId': '14574695992920015zm9_150393630...     None  ...     

    Beginner Advanced All                                              track  \
147        N        N   N  [Containers, Serverless, and Cloud, Infrastruc...   
151        N        N   N  [Containers, Serverless, and Cloud, Infrastruc...   
148        Y        Y   N  [DevOps and Pipelines, Development Tools, Java...   
166        Y        Y   N  [DevOps and Pipelines, Development Tools, Java...   
149        Y        N   N  [Containers, Serverless, and Cloud, Infrastruc...   
167        Y        N   N  [Containers, Serverless, and Cloud, Infrastruc...   
146        Y        N   N  [Database, Big Data, and Data Science, Infrast...   
15

In [184]:
#are duplicates indeed associated with both events?
# some are - but not all of them:
duplicates.loc[duplicates['code']=='BOF4977']


abstract  \
147  Microservices are independent—sure. Complex tr...   
151  Microservices are independent—sure. Complex tr...   

                                       attributevalues  catalog     code  \
147  [{'value': 'Intermediate', 'attributevalue_id'...      oow  BOF4977   
151  [{'value': 'Intermediate', 'attributevalue_id'...  codeone  BOF4977   

                event eventCode  \
147  Oracle OpenWorld     oow18   
151  Oracle OpenWorld     oow18   

                                                 files  length  \
147  [{'fileId': '1540345180625001DgUR', 'filename'...    45.0   
151  [{'fileId': '1540345180625001DgUR', 'filename'...    45.0   

                                          participants sponsors  ...    \
147  [{'speakerId': '14574694420080015I8e_150393630...     None  ...     
151  [{'speakerId': '14574694420080015I8e_150393630...     None  ...     

    Beginner Advanced All                                              track  \
147        N        N   N  [Containers, Serverless, and Cloud, Infrastruc...   
151        N        N   N  [Containers, Serverless, and Cloud, Infrastruc...   

    speaker_count instance_count room_capacity      day  \
147             1            1.0           303  Tuesday   
151             1            1.0           303  Tuesday   

                         room   time  
147  Moscone West - Room 2006  19:30  
151  Moscone West - Room 2006  19:30  

[2 rows x 31 columns]

In [185]:
# find all sessions that occur in both catalogs: set their catalog attribute to both
# set catalog="both" if session in duplicates with a different catalog value than the session's own catalog value
doubleCatalogSessions = pd.merge(ss, duplicates, on=['code'], how='inner').query('catalog_y != catalog_x')
doubleCatalogSessions[['code','catalog_x', 'catalog_y']] .head(20)



code catalog_x catalog_y
45   TRN5793       oow   codeone
47   TRN5793       oow   codeone
48   TRN5793   codeone       oow
50   TRN5793   codeone       oow
53   TRN5793       oow   codeone
55   TRN5793       oow   codeone
56   TRN5793   codeone       oow
58   TRN5793   codeone       oow
137  TRN4084       oow   codeone
139  TRN4084       oow   codeone
140  TRN4084   codeone       oow
142  TRN4084   codeone       oow
145  TRN4084       oow   codeone
147  TRN4084       oow   codeone
148  TRN4084   codeone       oow
150  TRN4084   codeone       oow
181  TRN4065       oow   codeone
183  TRN4065       oow   codeone
184  TRN4065   codeone       oow
186  TRN4065   codeone       oow

In [186]:
# all sessions in doubleCatalogSessions occur in both oow and code one session catalog
# time to update column catalog for all sessions in ss that have a code that occurs in doubleCatalogSessions['code']
ss.loc[ss['code'].isin(doubleCatalogSessions['code']),'catalog']='both'
ss.sort_values('catalog').head(50)

abstract  \
1169  In this session get a practical introduction t...   
1157  In this session learn to develop and deploy a ...   
1158  In this session learn about the latest feature...   
1164  One of the marquee features of Oracle Applicat...   
1167  In this session learn how to use the MySQL Doc...   
1168  In this session discover Oracle NoSQL Database...   
2337  During this tutorial, you will learn how to us...   
1183  In this lab learn to write an asynchronous ser...   
1192  Oracle Application Express is a low-code devel...   
1204  <b>PLEASE NOTE: YOU MUST BRING YOUR OWN LAPTOP...   
1221  Get hands-on and learn the features of the Pyt...   
1229  MySQL 8.0 added support for common table expre...   
1230  Oracle Cloud Infrastructure Container Engine f...   
1231  The Terraform Oracle Cloud Infrastructure prov...   
1238  One of the marquee features of Oracle Applicat...   
1240  In this session discover Oracle NoSQL Database...   
1154  Join this session for the State of the Dolphin...   
1143  Join this session for the State of the Dolphin...   
1112  MySQL Connector/Python provides three APIs to ...   
1111  MySQL X Dev API for PHP provides three APIs to...   
364   The value of a platform is in the solutions it...   
363   Are you moving to the cloud? Looking at contai...   
362   In this session the Oracle ACE team again take...   
361   Since Oracle announced support for some of its...   
360   Whether you're simulating automotive crashes o...   
346   Provisioning the hardware and software needed ...   
336   The value of a platform is in the solutions it...   
1241  In this session get a practical introduction t...   
1066  NASA’s Lunar Orbital Platform-Gateway is plann...   
329   Are you moving to the cloud? Looking at contai...   
1080  Container registries are evolving as container...   
317   In this session the Oracle ACE team again take...   
1084  Jenkins may be entering its teenage years, but...   
1094  This session presents a guide for enterprises ...   
1103  This session focuses on a modern DevOps approa...   
290   Since Oracle announced support for some of its...   
1067  Startup development teams are leveraging Oracl...   
250   Microservices can help teams bring change with...   
1253  In this lab learn to write an asynchronous ser...   
244   Whether you're simulating automotive crashes o...   
1450  There are substantial improvements in the MySQ...   
1451  MySQL 8 is the new kid on the block, and Perco...   
1454  MySQL analytics has been optimized to take ful...   
1503  In this session discover the power of MySQL in...   
1508  Need to provide REST APIs for your Oracle data...   
1509  The rapid adoption of enterprise cloud–based s...   
1513  The big data ecosystem has grown exponentially...   
1449  Looking for the latest news about MySQL 8.0 pe...   
1563  The latest developments and the enticing roadm...   
167   The migration from monolith to microservices h...   

                                        attributevalues catalog     code  \
1169  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL1706   
1157  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6325   
1158  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6328   
1164  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6326   
1167  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL1703   
1168  [{'value': 'Intermediate', 'attributevalue_id'...    both  HOL6302   
2337  [{'value': 'Beginner', 'attributevalue_id': '1...    both  TUT6303   
1183  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL4799   
1192  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6327   
1204  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6311   
1221  [{'value': 'Beginner', 'attributevalue_id': '1...    both  HOL6329   
1229  [{'value': 'Intermediate', 'attributevalue_id'...    both  HOL2986   
1230  [{'value': 'Beginner', 'attributevalue_id':

In [187]:
#Drop duplicates - identifying rows by their code
ss.drop_duplicates(subset=['code'], keep='first', inplace=True)

In [188]:
#hpw many sessions appear in both catalogs?
ss['catalog'].value_counts()

oow        1434
codeone     477
both        141
Name: catalog, dtype: int64

## Persist Pandas Dataframe - as single, consolidated, reshaped, enriched JSON file

One thing we want to be able to do with the data we gather, is to persist it for future use. We could store the data in a NoSQL database, a cloud storage service or simply as a local file. For now, let's do the latter: store the cleansed and reshaped data in a local JSON file for further enrichment, visualization and machine learning purposes.

In [189]:
ss.to_json("oow2018-sessions-wrangled.json", force_ascii=False)

In [190]:
dubss = pd.read_json("oow2018-sessions-wrangled.json")
dubss.head(20)

Advanced All Beginner Groundbreaker Ambassador Intermediate  \
0           Y   N        Y                        N            Y   
1           Y   N        Y                        N            Y   
10          Y   N        Y                        N            Y   
100         N   N        N                        N            N   
1000        Y   N        N                        N            Y   
1002        N   N        N                        N            Y   
1005        N   N        Y                        N            N   
1006        N   N        N                        N            Y   
1007        N   N        N                        N            Y   
1008        N   N        Y                        N            N   
1009        Y   N        N                        Y            N   
101         Y   N        Y                        N            Y   
1010        Y   N        N                        N            Y   
1011        N   N        N                        Y            Y   
1012        N   N        N                        N            Y   
1013        N   N        N                        N            Y   
1014        N   N        Y                        N            Y   
1015        N   N        N                        N            Y   
1016        Y   N        Y                        N            Y   
1017        N   N        Y                        N            Y   

     JavaOne Rockstar Oracle ACE Director Oracle Java Champion  \
0                   N                   N                    N   
1                   N                   N                    N   
10                  Y                   N                    N   
100                 N                   N                    N   
1000                N                   N                    Y   
1002                N                   N                    N   
1005                N                   Y                    N   
1006                N                   N                    N   
1007                Y                   N                    N   
1008                N                   N                    N   
1009                Y                   N                    Y   
101                 N                   N                    N   
1010                N                   N                    N   
1011                N                   N                    N   
1012                Y                   N                    Y   
1013                Y                   N                    Y   
1014                N                   N                    Y   
1015                N                   N                    N   
1016                N                   N                    N   
1017                N                   N                    N   

                                               abstract  \
0     What if you could benefit from new database qu...   
1     How does a real-world performance engineer int...   
10    In this session learn how to make client data ...   
100   In this session learn how to  transfer your st...   
1000  With Java Flight Recorder (JFR) and Java Missi...   
1002  Failure is not an option in services that need...   
1005  One of the most important factors influencing ...   
1006  The long-awaited Transport Layer Security (TLS...   
1007  Apart from the Java heap and the metaspace, wh...   
1008  Planning to build microservices? The best prac...   
1009  With the release of Java 9, our beloved mostly...   
101   In this session join development and product m...   
1010  Embracing microservice architecture in a new p...   
1011  Application development in the cloud has alrea...   
1012  We all have seen our share of bad code and som...   
1013  Java 10 has local variable type inference, but...   
1014  The selection of the right toolset to develop,...   
1015  Is it possible to create and consume a standar...   
1016  Coming into a GDPR-project where you’re suppos...   
1017  Int

If and when the previous cell lists 20 sessions, the data lake has been populated with the consolidated file *oow2018-sessions-wrangled.json* with all sessions - cleansed, deduplicated, enriched and ready for further processing.